In [1]:
import numpy as np
import pandas as pd
import json
import nltk
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import re
import math
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
def load_SNLI(path:str):
    with open(path, 'r') as f:
        # Read all lines in the file and parse them as JSON
        data = [json.loads(line.strip()) for line in f]
    return data

def load_SNLI_as_pandas(path:str):
    df = pd.read_json(path, lines=True)
    return df

In [3]:
#load the training data into a variable for future use
path_data = "SNLI_data/snli_1.0_train.jsonl"
snli_data = load_SNLI(path=path_data)

In [4]:
snli_data_pd = load_SNLI_as_pandas(path=path_data)

In [5]:
snli_data_pd.head(10)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
5,[contradiction],2267923837.jpg#2,contradiction,2267923837.jpg#2r1c,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,The kids are frowning,( ( The kids ) ( are frowning ) ),(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...
6,[contradiction],3691670743.jpg#0,contradiction,3691670743.jpg#0r1c,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy skates down the sidewalk.,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...
7,[entailment],3691670743.jpg#0,entailment,3691670743.jpg#0r1e,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy does a skateboarding trick.,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...
8,[neutral],3691670743.jpg#0,neutral,3691670743.jpg#0r1n,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy is wearing safety equipment.,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...
9,[neutral],4804607632.jpg#0,neutral,4804607632.jpg#0r1n,An older man sits with his orange juice at a s...,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man drinks his juice as he waits for ...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...


### Preprocessing functions

In [7]:
# Here to implement all functions for preprocessing the given data
def remove_duplicates(df:pd.DataFrame, subset:list[str]) -> pd.DataFrame:
    df_preprocessed = df.drop_duplicates(subset=subset)

    return df_preprocessed

def lower_tokenize_punct(lst:list[str]):
    stopwords_english = set(stopwords.words("english"))
    preprocessed_lst = []
    for sent in lst:
        lowercased_sent = sent.lower() # lowercase
        tokens_of_sent = word_tokenize(lowercased_sent) # tokenize
        clean_punctuation = [w for w in tokens_of_sent if w not in stopwords_english and w not in string.punctuation] # handle stopwords and punctuation

        preprocessed_lst.append(clean_punctuation)
    return preprocessed_lst
    
#lowercase, remove stopwords, tokenize --> Add new columns
def preprocess_dataF(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy(deep=True)
    premises = df["sentence1"].values
    hypotheses = df["sentence2"].values

    preprocessed_premises = lower_tokenize_punct(premises)
    preprocessed_hypotheses = lower_tokenize_punct(hypotheses)

    df_out["sentence1_preprocessed"] = preprocessed_premises
    df_out["sentence2_preprocessed"] = preprocessed_hypotheses

    return df_out

### Helpful Functions

In [30]:
def count_words_dict(df: pd.DataFrame) -> dict:
    out_count_dict = nltk.defaultdict(int)
    premises = df["sentence1_preprocessed"].values
    hypotheses = df["sentence2_preprocessed"].values

    merge = premises + hypotheses

    for sent in merge:
        for word in sent:
            out_count_dict[word] += 1
    return out_count_dict

def sort_dict(d: dict, reverse=True) -> dict:
    d_out = sorted(d.items(), key=lambda x: x[1] , reverse=reverse)
    return dict(d_out)

def remove_less_than_n_dict(d: dict, n:int) -> dict:
    new_dict = {k: v for k, v in d.items() if v >=n}
    return new_dict

In [13]:
print(f"Count of ALL samples : {len(snli_data_pd)}")

Count of ALL samples : 550152


In [10]:
subset_distinct = ["sentence1", "sentence2"]
snli_data_distinct = remove_duplicates(df=snli_data_pd, subset=subset_distinct)

In [12]:
print(f"Count of distinct samples : {len(snli_data_distinct)}")

Count of distinct samples : 549526


In [14]:
a = []
b = [1, 2, 3]
c = [2, 3, 555]
a.append(b)
a.append(c)
a

[[1, 2, 3], [2, 3, 555]]

In [15]:
snli_data_pd_preprocessed = preprocess_dataF(snli_data_distinct)

In [16]:
snli_data_pd_preprocessed.columns

Index(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1',
       'sentence1_binary_parse', 'sentence1_parse', 'sentence2',
       'sentence2_binary_parse', 'sentence2_parse', 'sentence1_preprocessed',
       'sentence2_preprocessed'],
      dtype='object')

In [17]:
snli_data_pd_preprocessed.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1_preprocessed,sentence2_preprocessed
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, training, horse, competition]"
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, diner, ordering, omelette]"
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, outdoors, horse]"
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,"[children, smiling, waving, camera]","[smiling, parents]"
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,"[children, smiling, waving, camera]","[children, present]"


In [22]:
snli_data_pd_preprocessed["annotator_labels"][1]

['contradiction']

In [23]:
c_dict = count_words_dict(snli_data_pd_preprocessed)

In [26]:
c_dict, len(c_dict)

(defaultdict(int,
             {'person': 35987,
              'horse': 7221,
              'jumps': 6628,
              'broken': 736,
              'airplane': 1219,
              'training': 734,
              'competition': 2594,
              'diner': 253,
              'ordering': 260,
              'omelette': 2,
              'outdoors': 9109,
              'children': 26545,
              'smiling': 8635,
              'waving': 1370,
              'camera': 11371,
              'parents': 1024,
              'present': 851,
              'kids': 10748,
              'frowning': 312,
              'boy': 58262,
              'jumping': 13405,
              'skateboard': 4720,
              'middle': 4580,
              'red': 43302,
              'bridge': 3216,
              'skates': 534,
              'sidewalk': 12819,
              'skateboarding': 1981,
              'trick': 3313,
              'wearing': 80985,
              'safety': 1904,
              'equipment': 2

In [29]:
a = [[1, 2], [2, 5, 6]]
b = [[1, 4, 5, 6]]
a + b 

[[1, 2], [2, 5, 6], [1, 4, 5, 6]]

In [31]:
sorted_dict = sort_dict(c_dict)

In [38]:
sorted_dict_10 = remove_less_than_n_dict(d=sorted_dict, n=10)

In [39]:
len(sorted_dict_10)

12398

In [41]:
premises11 = list(snli_data_pd_preprocessed["sentence1_preprocessed"].values)
hypotheses11 = list(snli_data_pd_preprocessed["sentence2_preprocessed"].values)


merge_both_as_tuple = list(zip(premises11, hypotheses11))
print(merge_both_as_tuple[:10])

[(['person', 'horse', 'jumps', 'broken', 'airplane'], ['person', 'training', 'horse', 'competition']), (['person', 'horse', 'jumps', 'broken', 'airplane'], ['person', 'diner', 'ordering', 'omelette']), (['person', 'horse', 'jumps', 'broken', 'airplane'], ['person', 'outdoors', 'horse']), (['children', 'smiling', 'waving', 'camera'], ['smiling', 'parents']), (['children', 'smiling', 'waving', 'camera'], ['children', 'present']), (['children', 'smiling', 'waving', 'camera'], ['kids', 'frowning']), (['boy', 'jumping', 'skateboard', 'middle', 'red', 'bridge'], ['boy', 'skates', 'sidewalk']), (['boy', 'jumping', 'skateboard', 'middle', 'red', 'bridge'], ['boy', 'skateboarding', 'trick']), (['boy', 'jumping', 'skateboard', 'middle', 'red', 'bridge'], ['boy', 'wearing', 'safety', 'equipment']), (['older', 'man', 'sits', 'orange', 'juice', 'small', 'table', 'coffee', 'shop', 'employees', 'bright', 'colored', 'shirts', 'smile', 'background'], ['older', 'man', 'drinks', 'juice', 'waits', 'daught

In [63]:
def co_occurence_two_words(df: pd.DataFrame, w_i, w_j) -> int:
    premises = list(df["sentence1_preprocessed"].values)
    hypotheses = list(df["sentence2_preprocessed"].values)
    count = 0

    merge_both_as_tuple = list(zip(premises, hypotheses))
    for p, h in merge_both_as_tuple:
        if w_i in p and w_j in p:
            count += 1
        if w_i in h and w_j in h:
            count += 1
    return count

def calculate_pmi(df:pd.DataFrame, N: int, counts:dict, w_i, w_j) -> float:
    co_occ_value = co_occurence_two_words(df=df, w_i= w_i, w_j=w_j)
    count_wi = counts[w_i]
    count_wj = counts[w_j]

    numerator = float(N * co_occ_value)
    denominator = float(count_wi * count_wj)
    if numerator == 0:
        return 0
    
    return math.log2(numerator/denominator)

def tool(df: pd.DataFrame, N: int, counts:dict, w:str) -> dict:
    out_dict = nltk.defaultdict(int)
    counter = 0
    for k,v in counts.items():
        pmi = calculate_pmi(df=df, N=N, counts=counts, w_i=w, w_j=k)
        out_dict[k] = pmi
        print(f"iter {counter} : {k}, PMI: {pmi}")
        counter += 1
        
    #rank by PMI
    return dict(sorted(out_dict.items(), key=lambda x: x[1], reverse=True)) 

In [43]:
hypothese = list(snli_data_pd_preprocessed["sentence2_preprocessed"].values)
hypothese

[['person', 'training', 'horse', 'competition'],
 ['person', 'diner', 'ordering', 'omelette'],
 ['person', 'outdoors', 'horse'],
 ['smiling', 'parents'],
 ['children', 'present'],
 ['kids', 'frowning'],
 ['boy', 'skates', 'sidewalk'],
 ['boy', 'skateboarding', 'trick'],
 ['boy', 'wearing', 'safety', 'equipment'],
 ['older', 'man', 'drinks', 'juice', 'waits', 'daughter', 'get', 'work'],
 ['boy', 'flips', 'burger'],
 ['elderly', 'man', 'sits', 'small', 'shop'],
 ['women', 'hugging', 'vacation'],
 ['women', 'sleeping'],
 ['women', 'showing', 'affection'],
 ['people', 'eating', 'omelettes'],
 ['people', 'sitting', 'desks', 'school'],
 ['diners', 'restaurant'],
 ['man', 'drinking', 'juice'],
 ['two', 'women', 'restaurant', 'drinking', 'wine'],
 ['man', 'restaurant', 'waiting', 'meal', 'arrive'],
 ['blond', 'man', 'getting', 'drink', 'water', 'fountain', 'park'],
 ['blond',
  'man',
  'wearing',
  'brown',
  'shirt',
  'reading',
  'book',
  'bench',
  'park'],
 ['blond', 'man', 'drinking', 

In [44]:
co_occurence_two_words(snli_data_pd_preprocessed, "men", "women") 

2292

In [46]:
N = len(snli_data_pd_preprocessed)
N

549526

In [47]:
calculate_pmi(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_10 , w_i="men", w_j="women")

-0.7356023133153547

In [62]:
sorted_dict_100 = remove_less_than_n_dict(d=sorted_dict, n=100)

In [58]:
a = tool(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_100, w="men")

iter 0 : man, PMI: -3.2385865920726484
iter 1 : woman, PMI: -2.6553466567906883
iter 2 : two, PMI: 1.061879277119484
iter 3 : people, PMI: -3.7909707463173437
iter 4 : wearing, PMI: -0.9188072264204178
iter 5 : young, PMI: -1.2088703873855706
iter 6 : men, PMI: 3.158835804448575
iter 7 : playing, PMI: -0.5372104910824801
iter 8 : girl, PMI: -5.2829751332914405
iter 9 : boy, PMI: -4.7105701297948785
iter 10 : white, PMI: -1.1964448106691468
iter 11 : shirt, PMI: -2.4662530152637507
iter 12 : black, PMI: -1.3334264798543405
iter 13 : dog, PMI: -4.419844776558899
iter 14 : sitting, PMI: -0.6813692405430056
iter 15 : blue, PMI: -1.3839477374633047
iter 16 : standing, PMI: -0.28903588281078374
iter 17 : red, PMI: -1.3955548766299481
iter 18 : group, PMI: -0.03189530857981145
iter 19 : walking, PMI: -1.1599584043556475
iter 20 : outside, PMI: -0.7683631407014834
iter 21 : street, PMI: -0.9469990557432056
iter 22 : person, PMI: -5.87639513266816
iter 23 : front, PMI: -0.6621386500996895
iter 

In [59]:
a

{'men': 3.158835804448575,
 'suits': 1.889829874813191,
 'hats': 1.8580735212067925,
 'shirts': 1.6214070224362453,
 'jackets': 1.4993993434462998,
 'uniforms': 1.4123756469501676,
 'two': 1.061879277119484,
 'instruments': 1.0383415293630267,
 '2': 1.0082031337070203,
 'three': 0.9676329251420207,
 'hard': 0.8602576943794367,
 'five': 0.8483853321231006,
 'fighting': 0.8401667592631653,
 'six': 0.8337205277697998,
 'talk': 0.79145480079279,
 'four': 0.7551023329437406,
 'safety': 0.7318397018860491,
 'working': 0.7082210390612883,
 'stand': 0.6508866370627319,
 'roof': 0.6440063700362199,
 '3': 0.6304488126323925,
 'sit': 0.6191078131292388,
 'work': 0.6189304479427873,
 'beer': 0.5700790787414447,
 'horses': 0.5290500665381099,
 'arts': 0.46776780899043946,
 'one': 0.42835828427929146,
 'talking': 0.4091979132141175,
 'boat': 0.39450783397144834,
 'business': 0.39224896668850695,
 'chairs': 0.3273582553120473,
 'fishing': 0.31743273689479756,
 'costumes': 0.3025010678577494,
 'togeth

In [55]:
a

{'men': 3.158835804448575,
 'two': 1.061879277119484,
 'three': 0.9676329251420207,
 'four': 0.7551023329437406,
 'working': 0.7082210390612883,
 'stand': 0.6508866370627319,
 'sit': 0.6191078131292388,
 'work': 0.6189304479427873,
 'one': 0.42835828427929146,
 'talking': 0.4091979132141175,
 'boat': 0.39450783397144834,
 'together': 0.2928234960968003,
 'several': 0.13402609187589395,
 'look': 0.05748377783064129,
 'play': 0.037339729818168695,
 'construction': 0.024792520809610516,
 'group': -0.03189530857981145,
 'basketball': -0.13418584252034693,
 'table': -0.14415244073274816,
 'watch': -0.1830852026722097,
 'building': -0.217659160171574,
 'dressed': -0.22650516431447357,
 'stage': -0.22960694840336884,
 'walk': -0.241553781825142,
 'standing': -0.28903588281078374,
 'around': -0.3282843438769654,
 'music': -0.3537341629435878,
 'game': -0.35731510310278386,
 'side': -0.4240155516458569,
 'room': -0.4454409940624005,
 'orange': -0.46567809248429537,
 'playing': -0.53721049108248

In [56]:
b = tool(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_10, w="women", limit=200)

iter 0 : man, PMI: -2.501759009837856
iter 1 : woman, PMI: -3.226451553477119
iter 2 : two, PMI: 0.9226235061038228
iter 3 : people, PMI: -3.537669984045648
iter 4 : wearing, PMI: -0.9199225225129869
iter 5 : young, PMI: -0.7981992113378672
iter 6 : men, PMI: -0.7356023133153547
iter 7 : playing, PMI: -1.5137487574749535
iter 8 : girl, PMI: -4.062264618892472
iter 9 : boy, PMI: -4.159337935077015
iter 10 : white, PMI: -1.3993664626439855
iter 11 : shirt, PMI: -2.535985437968601
iter 12 : black, PMI: -1.321066206291954
iter 13 : dog, PMI: -4.373225554941742
iter 14 : sitting, PMI: -0.42160945572226843
iter 15 : blue, PMI: -1.748926461291024
iter 16 : standing, PMI: -0.6207747846647663
iter 17 : red, PMI: -1.4818007738519006
iter 18 : group, PMI: -0.07161765964654923
iter 19 : walking, PMI: -0.26339305169628013
iter 20 : outside, PMI: -0.8364001535623132
iter 21 : street, PMI: -0.35590686276049666
iter 22 : person, PMI: -4.4784724068356585
iter 23 : front, PMI: -0.8081066586316253
iter 2

In [57]:
b

{'women': 3.983784760219459,
 'two': 0.9226235061038228,
 'dancing': 0.904901900424744,
 'walk': 0.8998019130549946,
 'sit': 0.8930064916002304,
 'three': 0.7334620225268811,
 'talking': 0.5331834153633819,
 'together': 0.4912923193555941,
 'four': 0.32196151279458485,
 'table': 0.3011834878530402,
 'asian': 0.16777000472224365,
 'stand': 0.1500867505409876,
 'look': 0.12373851309436806,
 'bench': 0.03310739723648343,
 'store': 0.032337924296353196,
 'one': -0.007869649670569316,
 'smiling': -0.03035222108178292,
 'group': -0.07161765964654923,
 'dressed': -0.0902398826096017,
 'older': -0.09943712593025274,
 'several': -0.11451990865640571,
 'stage': -0.16037394871787525,
 'sidewalk': -0.259474487836655,
 'walking': -0.26339305169628013,
 'blond': -0.31292827627761277,
 'street': -0.35590686276049666,
 'bus': -0.38305592554898416,
 'sitting': -0.42160945572226843,
 'camera': -0.4329080334497591,
 'dark': -0.4419697320043399,
 'carrying': -0.5043878122572143,
 'restaurant': -0.51316895